<a href="https://colab.research.google.com/github/sushanttwayana/Text2HumanFace_Generation/blob/main/Celeba_Dat_Generation_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'celeba-10000-text:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5343246%2F8876678%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240706%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240706T174955Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D76e13c8c9df3ac391eed15c7cf5c84310586b801ba8ef5066f1955e82cb958c108096754d804c004c0e989af415d0b993361da0b6bb85eae0e8724b41ddae7fa3080add095dcd37aad71cdbd4e5e5cfe2d0bc439925a2d5e34698b98142f12e992cd984c01e954ff5f66778083ccba06bc0d203f89fd6af7f0b777fa8327275f221ddabf792feb582f87a4993d36ca54382a33f2f9cf5a4e5327f1b85a756c2631633bb59e100b706e8c1fe99678f75b9740abe2a510ef56a40b9cbc5ac90cf67a058539f20fef4e363c4cf9e77d69046a5510680c53cea86c43f0113aa45fd2ffe028c264888b5cd727c91382ce0cfbae7fbeb0522f9fb677e57daeaa846120,caption-pickle:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5343697%2F8878042%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240706%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240706T174955Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6a0ee722eb4d740237046f0c1217a53b283178d752a809144d87e9d5005928da39cb0616437adc90aa613927e2b08860496799e6f917b7ba17acaa02841d125a061a4c754cc96bf13eff8695d67bf49f242dd72d53e91802379b65a79e958d67fda8720cf2a0bcc999adbe65172a1b0673ce577cc03d9430e067c48832df809b576ae32f00637ec039010505ac75caceddef6fc45b23669a5b44540c44124a36618c3c1e4de1afc5ae3be133a5316f0ab89480eb240e6d98296db4876a35aeb912ffbd22ea05ab4de21b7ac9d71302564a684418e68b6e86c7c3db54f4a8e62119f78042326d1d0ab6e210c381927b1a3e92570d4acfdc3614a2c579020ec5b9,celeba-text-10000:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5344168%2F8879293%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240706%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240706T174955Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6dd9152f1cab52ed1b838410812242d2e061fc99055bd79bfc6615ccd3e6c25e7e561a0924508a5037d4fb9d5c7aef718be499eca5059552a7689d3ed42e850330973834fdb46be1d43082176b3483e0a0ed1124213192a67871a92953e06e7edc21bdc408b58442ac7c01761b6feb87547c9771cb9c8ba92b0c53775e28b19013c29d671637ebac94f9b6eb453dceb18c38b6928d953ab054d925aa829fc002e72a762e8a971bcc5bbed7b1a91185c72b49a868baa939a17b10d5beed2722a1ff1e88aead62276283a0941021a84081113d55a8a7b00f3be291a6e994947e6d1c71fc04b36eca4fe5cbfb439e9e598c65af0db0aa18f52e3863518c5842e94f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import os

celeba = os.listdir("/kaggle/input/celeba-10000-text/selected_images_celeba_10000")

celeba.sort()

print(celeba[0])
print(len(celeba))

000002.txt
10000


In [ ]:
import os

# Directory containing the files
directory_path = "/kaggle/input/celeba-10000-text/selected_images_celeba_10000"

# List all files in the directory
file_names = os.listdir(directory_path)

# Iterate over each file in the directory
for file_name in file_names:
    # Full path to the file
    file_path = os.path.join(directory_path, file_name)

    # Read the input description from the file
    with open(file_path, 'r') as file:
        description = file.read()

    # Split the description by periods followed by a space
    sentences = description.split('. ')

    # Remove leading and trailing whitespace from each sentence
    sentences = [sentence.strip() for sentence in sentences]

    # Join the sentences with a period and a newline
    formatted_description = '.\n'.join(sentences)

    # Add a period at the end of the last sentence if it's missing
    if not formatted_description.endswith('.'):
        formatted_description += '.'

    # Output file path
    output_file_path = os.path.join('formatted_descriptions', file_name)

    # Create output directory if it doesn't exist
    os.makedirs('formatted_descriptions', exist_ok=True)

    # Write the formatted description to a new file
    with open(output_file_path, 'w') as file:
        file.write(formatted_description)

print(f"Formatted description written to {output_file_path}")


Formatted description written to formatted_descriptions/051270.txt
Formatted description written to formatted_descriptions/048669.txt
Formatted description written to formatted_descriptions/143838.txt
Formatted description written to formatted_descriptions/196256.txt
Formatted description written to formatted_descriptions/020936.txt
Formatted description written to formatted_descriptions/187632.txt
Formatted description written to formatted_descriptions/028086.txt
Formatted description written to formatted_descriptions/009580.txt
Formatted description written to formatted_descriptions/150167.txt
Formatted description written to formatted_descriptions/041810.txt
Formatted description written to formatted_descriptions/066150.txt
Formatted description written to formatted_descriptions/067751.txt
Formatted description written to formatted_descriptions/182333.txt
Formatted description written to formatted_descriptions/007478.txt
Formatted description written to formatted_descriptions/145598

In [ ]:
import os
import zipfile

# Directory containing the files
directory_path = "/kaggle/input/celeba-10000-text/selected_images_celeba_10000"

# List all files in the directory
file_names = os.listdir(directory_path)

# Create a directory for the formatted descriptions
output_dir = 'formatted_descriptions'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each file in the directory
for file_name in file_names:
    # Full path to the file
    file_path = os.path.join(directory_path, file_name)

    # Read the input description from the file
    with open(file_path, 'r') as file:
        description = file.read()

    # Split the description by periods followed by a space
    sentences = description.split('. ')

    # Remove leading and trailing whitespace from each sentence
    sentences = [sentence.strip() for sentence in sentences]

    # Join the sentences with a period and a newline
    formatted_description = '.\n'.join(sentences)

    # Add a period at the end of the last sentence if it's missing
    if not formatted_description.endswith('.'):
        formatted_description += '.'

    # Output file path
    output_file_path = os.path.join(output_dir, file_name)

    # Write the formatted description to a new file
    with open(output_file_path, 'w') as file:
        file.write(formatted_description)

print(f"Formatted description written to {output_file_path}")

# Create a zip file containing all the formatted files
zip_file_path = 'formatted_descriptions.zip'
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_dir))

print(f"All formatted descriptions have been zipped into {zip_file_path}")


Formatted description written to formatted_descriptions/200944.txt
All formatted descriptions have been zipped into formatted_descriptions.zip


# Check pickle File
#

In [ ]:
import pickle

# Load the pickle file
file_path = '/kaggle/input/caption-pickle/captions_DAMSM2.pickle'
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Inspect the loaded object
print("Type of the loaded object:")
print(type(data))

# If the object is a dictionary, list its keys
if isinstance(data, dict):
    print("\nKeys in the dictionary:")
    for key in data.keys():
        print(key)

# If the object is a list, print its length and the type of its first element
elif isinstance(data, list):
    print(f"\nLength of the list: {len(data)}")
    if len(data) > 0:
        print(f"Type of the first element: {type(data[0])}")

# Print the first few elements/entries to get an idea of its structure
print("\nFirst few elements/entries:")
for i, element in enumerate(data if isinstance(data, list) else data.items()):
    if i == 0:  # Limit the output to the first 5 elements/entries
        print(len(element))

#     else:
#         break

Type of the loaded object:
<class 'list'>

Length of the list: 4
Type of the first element: <class 'list'>

First few elements/entries:
205320


In [ ]:
import pickle

# Load the pickle file
file_path = '/kaggle/input/caption-pickle/captions_DAMSM_bird.pickle'
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Inspect the loaded object
print("Type of the loaded object:")
print(type(data))

# If the object is a dictionary, list its keys
if isinstance(data, dict):
    print("\nKeys in the dictionary:")
    for key in data.keys():
        print(key)

# If the object is a list, print its length and the type of its first element
elif isinstance(data, list):
    print(f"\nLength of the list: {len(data)}")
    if len(data) > 0:
        print(f"Type of the first element: {type(data[0])}")

# Print the first few elements/entries to get an idea of its structure
print("\nFirst few elements/entries:")
for i, element in enumerate(data if isinstance(data, list) else data.items()):
    if i == 0:  # Limit the output to the first 5 elements/entries
        print(len(element))

#     else:
#         break

Type of the loaded object:
<class 'list'>

Length of the list: 4
Type of the first element: <class 'list'>

First few elements/entries:
88550


# UNIQUE VOCABULARIES

In [ ]:
import os

# Directory containing the text files
directory_path = "/kaggle/input/celeba-10000-text/selected_images_celeba_10000"

# Initialize an empty set to store unique words
unique_words = set()

# List all files in the directory
file_names = os.listdir(directory_path)

# Iterate over each file in the directory
for file_name in file_names:
    # Full path to the file
    file_path = os.path.join(directory_path, file_name)

    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Split the content into words and add to the set
    words = content.split()
    unique_words.update(words)

# Get the number of unique words
num_unique_words = len(unique_words)

# Print the results
print(f"Number of unique words: {num_unique_words}")
print("Unique words:")
print(unique_words)

# Optionally, save the unique words to a file
with open('unique_vocabulary.txt', 'w') as file:
    for word in unique_words:
        file.write(word + '\n')

print("Unique vocabulary written to unique_vocabulary.txt")


Number of unique words: 106
Unique words:
{'bald.', 'bags', 'female', 'makeup.', 'cheerful.', 'cheeks.', 'joyful', 'hair', 'open.', 'necklace.', 'cheekbones.', 'blonde', 'seems', 'hairline.', 'dense', 'sideburns.', 'looks', 'oval', 'lipstick.', 'light', 'pointy', 'expression.', 'delighted.', 'thin', 'brown', 'mustache.', 'chubby.', 'rosy', 'neutral.', 'slim.', 'mouth', 'skin.', 'sad', 'his', 'round', 'stays', 'bangs.', 'pale', 'curly', 'facial', 'gray', 'unhappy', 'open-minded', 'attractive.', 'calm', 'male', 'nose.', 'hair.', 'old.', 'smiling.', 'wearing', 'hairs.', 'and', 'receding', 'small', 'He', 'feeling', 'on', 'clean-shaven', 'an', 'happy.', 'under', 'arched', 'a', 'is', 'depressed.', 'She', 'necktie.', 'earrings.', 'sides.', 'bushy', 'sad.', 'straight', 'fair-minded.', 'big', 'narrow', 'down.', 'closed.', 'This', 'no', 'goatee', 'wavy', 'bare', 'heavy', 'person', 'high', 'happy', 'black', 'chin.', 'rounded', 'her', 'eyes.', 'eyeglasses.', 'has', 'lips.', 'beard.', 'hat.', 'flat

In [ ]:
import os
import random
import string

# Function to generate random words
def generate_random_word(length=6):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

# Load the existing unique vocabulary from the file
existing_vocab_file = 'unique_vocabulary.txt'
with open(existing_vocab_file, 'r') as file:
    unique_words = set(file.read().split())

# Number of new words to add
num_new_words = 5345

# Generate new unique words and add them to the existing vocabulary
new_words = set()
while len(new_words) < num_new_words:
    new_word = generate_random_word()
    if new_word not in unique_words:
        new_words.add(new_word)

# Combine the existing vocabulary with the new words
updated_unique_words = unique_words.union(new_words)

# Save the updated vocabulary to a new file
updated_vocab_file = 'updated_unique_vocabulary.txt'
with open(updated_vocab_file, 'w') as file:
    for word in sorted(updated_unique_words):
        file.write(word + '\n')

print(f"Updated unique vocabulary written to {updated_vocab_file}")


Updated unique vocabulary written to updated_unique_vocabulary.txt


In [ ]:
print(len(updated_unique_words))

5451


In [ ]:
print(len(new_words))

5345


In [ ]:
print(new_words)

{'ohrdrr', 'gakxia', 'aeruyx', 'cosglx', 'zjndxs', 'uupajf', 'ojiuhi', 'aiisgm', 'wvzrqe', 'aoqlcq', 'pqdmhw', 'jzjzqf', 'zwxllb', 'qauhkv', 'gkvdwu', 'bxiwun', 'elkxpx', 'kdcdcu', 'rflezq', 'rzucpg', 'fetwxo', 'sxlimo', 'etgkln', 'alipwm', 'gkovtx', 'ivknin', 'adghjk', 'xhfinf', 'rwueeu', 'wuoxzg', 'ekxqgu', 'qtrcle', 'ephvwo', 'zykqew', 'gvfeiy', 'rxiigp', 'eyjjph', 'qvkghg', 'bpkzny', 'hlqdbw', 'qrjvhk', 'acdldh', 'ntvhjq', 'nhtmgv', 'sdpdwr', 'nbgypu', 'buwnex', 'ahbtyf', 'uwrybr', 'opkjin', 'pamums', 'cjbjcf', 'jcdlna', 'grvwic', 'ewlxjc', 'hximjm', 'eljclw', 'cqyfau', 'vfmtrd', 'xblzoa', 'avuauv', 'hvycfc', 'nftxzd', 'ybgmoc', 'ssafyp', 'rwderv', 'peabws', 'aabohe', 'bvsyjb', 'qrcvxa', 'fpvzdx', 'jhzqdh', 'brqriz', 'qikssg', 'olqahb', 'tynzom', 'hrwauf', 'tgcmmr', 'expebb', 'osuikh', 'feosnq', 'tgbfxk', 'erpsbs', 'gnwbip', 'xnklht', 'tyaxoz', 'bsqexc', 'wsdbsk', 'uhvisz', 'xyhets', 'tloyqj', 'aqnopv', 'cpojlt', 'xxsuip', 'ccrvug', 'avqvjn', 'xlsasz', 'ybxujl', 'lkzuoe', 'ygnsew',

In [ ]:
for i in new_words :

    for j in range(10):
        print(i)
        break

    break

ohrdrr


In [ ]:
import os
import random
import string

# Function to generate random words
def generate_random_word(length=6):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

# Load the existing unique vocabulary from the file
existing_vocab_file = 'unique_vocabulary.txt'
with open(existing_vocab_file, 'r') as file:
    unique_words = set(file.read().split())

# Number of new words to add
num_new_words = 5345

# Generate new unique words and add them to the existing vocabulary
new_words = set()
while len(new_words) < num_new_words:
    new_word = generate_random_word()
    if new_word not in unique_words:
        new_words.add(new_word)

# Save the new words 10 per line in a new file
new_words_file = 'new_unique_vocabulary.txt'
with open(new_words_file, 'w') as file:
    new_words_list = list(new_words)
    for i in range(0, len(new_words_list), 10):
        line_words = new_words_list[i:i + 10]
        file.write(' '.join(line_words) + '\n')

print(f"New unique words written to {new_words_file}")


New unique words written to new_unique_vocabulary.txt
